# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 87, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=87, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)

In [6]:
df = df[:-1].copy()

In [7]:
df.loc[1, 'Description'] ='Sable rouge'

In [8]:
df.query('ID=="F201"')

,ID,X,Y,Z,Litho_top,Description,Litho_base,Diam_for,Long_for
0,F201,152890.245758,122596.47426,101.926886,0.6,Remblais avec briques de construction couleur ...,1.2,75.0,2.4
1,F201,152890.245758,122596.47426,101.926886,1.2,Sable rouge,2.4,75.0,2.4


In [9]:
#df.drop(index=df.query('ID=="F201"').index, inplace=True)

## Create Boreholes from dataframes or files

In [10]:
bhs_df=[df] # list of dataframes

In [11]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon='fr', #Lexicon.default(),
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Long_for')


Dataframe 0 processing...
|__ID:'F201'
Summary : [Striplog(2 Intervals, start=0.6, stop=2.4)]
{0: {'description': 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7fafb0e92460>, 'base': <core.orm.PositionOrm object at 0x7fafb0e92100>}, 1: {'description': 'Sable rouge', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7fafb0ec23a0>, 'base': <core.orm.PositionOrm object at 0x7fafb0e9c340>}}

|__ID:'F205'
Error : No lithology matching with 'Couche de boue en surface liée à l'intervention de Géosonda' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{2: {'description': 'sable blanc', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7fafb0f26ac0>, 'base': <core.orm.PositionOrm object at 0x7fafb0ed85e0>}, 3: {'description': 'Limon brun ', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7fafb0e9c520>, 'base': <core.orm.PositionOrm object at

End of the process : 15 unique ID found


/home/su530201/PycharmProjects/GSDMA/utils/io.py:518: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/su530201/PycharmProjects/GSDMA/utils/io.py:518: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies I

In [12]:
components

{0: Component({'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'gris verdâtre'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'colour': 'verdâtre', 'grainsize': 'grossiers'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'verdâtres'}),
 7: Component({'lithology': 'limon'}),
 8: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'sableuse'}),
 9: Component({'grainsize': 'grossiers'}),
 10: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 11: Component({'lithology': 'remblais', 'grainsize': 'grossiers'}),
 12: Component({'lithology': 'limons', 'colour': 'noirs'}),
 13: Component({'lithology': 'remblais', 'modifier': 'sableux', 'colour': 'noirs gris'}),
 14: Component({'material': 'scories', 'grainsi

In [13]:
memoris_components = [Component({'lithology': 'remblais'}),
              Component({'lithology': 'limon'}),
              Component({'lithology': 'schistes'}),
              Component({'lithology': 'sable'}),
              Component({'lithology': 'anthropique'})
              ]
list_of_decors=[]
hatches = ['=', 'x', '.', 's', '=']
colours = ['#888888', '#882222', '#AAAAAA', '#CC22CC', '#CC2222']

for i in range(len(memoris_components)):
    if hasattr(memoris_components[i], 'colour'):
        c = memoris_components[i].colour
    else:
        c=colours[i]
    
    if hasattr(memoris_components[i], 'hatch'):
        h = memoris_components[i].hatch
    else:
        h=hatches[i]
        
    d = {'color': c,
         'hatch': h,
         'component': memoris_components[i],
         'width': 3}
    decor = Decor(d)
    list_of_decors.append(decor)
    
memoris_legend = Legend(list_of_decors)

In [14]:
df.query('ID=="F205"')

,ID,X,Y,Z,Litho_top,Description,Litho_base,Diam_for,Long_for
2,F205,152887.693175,122594.619977,101.804167,0.0,Couche de boue en surface liée à l'interventio...,0.2,75.0,4.8
3,F205,152887.693175,122594.619977,101.804167,0.2,Limon brun,0.4,75.0,4.8
4,F205,152887.693175,122594.619977,101.804167,0.4,Remblais gris gravillonneux avec présence de d...,1.2,75.0,4.8
5,F205,152887.693175,122594.619977,101.804167,1.2,Présence d'eau avec remblais pollués (odeur fo...,2.4,75.0,4.8
6,F205,152887.693175,122594.619977,101.804167,2.4,Remblais grossiers saturés avec présence de pr...,3.2,75.0,4.8
7,F205,152887.693175,122594.619977,101.804167,3.2,Alluvions verdâtres argilo-sableux (odeur dans...,3.6,75.0,4.8
8,F205,152887.693175,122594.619977,101.804167,3.6,Retombées des strates supérieures - Limon verd...,4.8,75.0,4.8


In [15]:
boreholes

[<core.orm.BoreholeOrm>(Name=F201, Length=4.8, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F205, Length=33.6, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F208, Length=24.0, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F212, Length=24.0, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F207, Length=33.6, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F214, Length=33.6, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F217, Length=24.0, Diameter=75.0, Intervals=5),
 <core.orm.BoreholeOrm>(Name=F221, Length=2.8, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F223, Length=1.3, Diameter=75.0, Intervals=1),
 <core.orm.BoreholeOrm>(Name=F225, Length=33.6, Diameter=75.0, Intervals=7),
 <core.orm.BoreholeOrm>(Name=F224, Length=4.8, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F219, Length=3.0, Diameter=75.0, Intervals=2),
 <core.orm.BoreholeOrm>(Name=F220, Length=0.5, Diameter=75.0, Intervals=1),
 <cor

In [16]:
components

{0: Component({'lithology': 'sable', 'colour': 'rouge'}),
 1: Component({'lithology': 'remblais', 'colour': 'gris verdâtre'}),
 2: Component({'lithology': 'remblais', 'Pollutant': 'naphtalène'}),
 3: Component({'lithology': 'limon', 'colour': 'verdâtre', 'grainsize': 'grossiers'}),
 4: Component({'lithology': 'remblais', 'grainsize': 'grossiers', 'Pollutant': 'naphtalène'}),
 5: Component({'lithology': 'remblais', 'colour': 'gris'}),
 6: Component({'modifier': 'sableux', 'colour': 'verdâtres'}),
 7: Component({'lithology': 'limon'}),
 8: Component({'lithology': 'remblais', 'modifier': 'sableuse', 'colour': 'sableuse'}),
 9: Component({'grainsize': 'grossiers'}),
 10: Component({'lithology': 'limons', 'colour': 'verdâtre'}),
 11: Component({'lithology': 'remblais', 'grainsize': 'grossiers'}),
 12: Component({'lithology': 'limons', 'colour': 'noirs'}),
 13: Component({'lithology': 'remblais', 'modifier': 'sableux', 'colour': 'noirs gris'}),
 14: Component({'material': 'scories', 'grainsi

In [17]:
len(boreholes), len(components)

(15, 44)

## Create a project and save data in a database

In [18]:
remove('tmp_files/tfe_orm_db.db')

In [19]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [20]:
Base.metadata.create_all(engine)

2021-06-15 09:28:10,307 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-15 09:28:10,309 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-06-15 09:28:10,315 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-15 09:28:10,319 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-06-15 09:28:10,320 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-15 09:28:10,322 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Positions")
2021-06-15 09:28:10,323 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-15 09:28:10,324 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Positions")
2021-06-15 09:28:10,325 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-15 09:28:10,327 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-06-15 09:28:10,331 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-15 09:28:10,333 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-06-15 09:28:10,334 INFO sqlalchemy.engine.Engine 

In [21]:
Session = sessionmaker(bind=engine)
session = Session()

In [22]:
p = Project(session, legend=memoris_legend)
p.add_components(components)

2021-06-15 09:28:10,432 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-15 09:28:10,440 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-06-15 09:28:10,441 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ()
Decor({'_colour': '#888888', 'hatch': '=', 'component': Component({'lithology': 'remblais'}), 'width': 3.0})
Decor({'_colour': '#882222', 'hatch': 'x', 'component': Component({'lithology': 'limon'}), 'width': 3.0})
Decor({'_colour': '#aaaaaa', 'hatch': '.', 'component': Component({'lithology': 'schistes'}), 'width': 3.0})
Decor({'_colour': '#cc22cc', 'hatch': 's', 'component': Component({'lithology': 'sable'}), 'width': 3.0})
Decor({'_colour': '#cc2222', 'hatch': '=', 'component': Component({'lithology': 'anthropique'}), 'width': 3.0})
2021-06-15 09:28:10,463 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?

Add boreholes into the project

In [23]:
for bh in boreholes:
    p.add_borehole(bh)

2021-06-15 09:28:10,508 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, length, diameter) VALUES (?, ?, ?)
2021-06-15 09:28:10,509 INFO sqlalchemy.engine.Engine [generated in 0.00164s] ('F201', 4.8, 75.0)
2021-06-15 09:28:10,515 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-06-15 09:28:10,516 INFO sqlalchemy.engine.Engine [generated in 0.00140s] ((0, 0.6, 0.6, 0.6, 152890.245758, 122596.47426), (1, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (2, 1.2, 1.2, 1.2, 152890.245758, 122596.47426), (3, 2.4, 2.4, 2.4, 152890.245758, 122596.47426))
2021-06-15 09:28:10,520 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, description, top_id, base_id) VALUES (?, ?, ?, ?, ?, ?)
2021-06-15 09:28:10,521 INFO sqlalchemy.engine.Engine [generated in 0.00119s] ((0, 'F201', 0, 'Remblais avec briques de construction couleur orange, gris, verdâtre  ', 0, 1), (1, 'F201', 1, 'Sable rouge', 2

TypeError: Cannot create a legend for empty component

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook